Importieren, aller benötigten Packages und einlesen der Trainings-, Test- und Überprüfungsdaten

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

datensatz = pd.read_csv("./foot-traffic-wue/train.csv")



Trainingsdaten nach Fehlern und unrelevanten Daten bereinigen

In [5]:
datensatz_f = datensatz[
    ~datensatz['incidents'].isin(['laser_failure'])
    ]

Markierung der Feiertage im Datensatz

In [6]:
# Liste der Feiertage
feiertage = [
    '2019-04-19', '2019-04-22', '2019-05-01', '2019-05-30', '2019-06-10', '2019-06-20', '2019-08-15',
    '2019-10-03', '2019-11-01', '2019-12-25', '2019-12-26', '2020-01-01', '2020-01-06', '2020-04-10',
    '2020-04-13', '2020-05-01', '2020-05-21', '2020-06-01', '2020-06-11', '2020-08-15', '2020-10-03',
    '2020-11-01', '2020-12-25', '2020-12-26', '2021-01-01', '2021-01-06', '2021-04-02', '2021-04-05',
    '2021-05-01', '2021-05-13', '2021-05-24', '2021-06-03', '2021-08-15', '2021-10-03', '2021-11-01',
    '2021-12-25', '2021-12-26', '2022-01-01', '2022-01-06', '2022-04-15', '2022-04-18', '2022-05-01',
    '2022-05-26', '2022-06-06', '2022-06-16', '2022-08-15', '2022-10-03', '2022-11-01', '2022-12-25',
    '2022-12-26', '2023-01-01', '2023-01-06', '2023-04-07', '2023-04-10', '2023-05-01', '2023-05-18',
    '2023-05-29', '2023-06-08', '2023-08-15', '2023-10-03', '2023-11-01', '2023-12-25', '2023-12-26',
    '2024-01-01', '2024-01-06', '2024-03-29', '2024-04-01', '2024-05-01', '2024-05-09', '2024-05-20',
    '2024-05-30', '2024-08-15'
]

# Konvertiere die Feiertage zu einem Set für eine schnellere Suche
feiertage_set = set(feiertage)

# Erstelle die neue Spalte "is_feiertag"
datensatz_f['is_feiertag'] = datensatz_f['date'].apply(lambda x: 'Ja' if x in feiertage_set else 'Nein')


/var/folders/by/wwdhnctx57b_vhf5wxchq4h00000gn/T/ipykernel_12610/4282932123.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datensatz_f['is_feiertag'] = datensatz_f['date'].apply(lambda x: 'Ja' if x in feiertage_set else 'Nein')


Aufteilung der Spalte date in year, month und day 

In [7]:
# Konvertiere die Spalte 'date' in ein Datetime-Format
datensatz_f['date'] = pd.to_datetime(datensatz_f['date'])

# Zerlege die Spalte in 'year', 'month' und 'day'
datensatz_f['year'] = datensatz_f['date'].dt.year
datensatz_f['month'] = datensatz_f['date'].dt.month
datensatz_f['day'] = datensatz_f['date'].dt.day


/var/folders/by/wwdhnctx57b_vhf5wxchq4h00000gn/T/ipykernel_12610/3628733809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datensatz_f['date'] = pd.to_datetime(datensatz_f['date'])
/var/folders/by/wwdhnctx57b_vhf5wxchq4h00000gn/T/ipykernel_12610/3628733809.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datensatz_f['year'] = datensatz_f['date'].dt.year
/var/folders/by/wwdhnctx57b_vhf5wxchq4h00000gn/T/ipykernel_12610/3628733809.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Zusammenführen der Wetterbedingungen, welche in Tag und Nacht unterteilt sind 

In [8]:
# Werte zusammenführen
datensatz_f['weather_condition'] = datensatz_f['weather_condition'].replace({
    'partly-cloudy-day': 'partly-cloudy',
    'partly-cloudy-night': 'partly-cloudy',
    'clear-day': 'clear',
    'clear-night': 'clear'
})

/var/folders/by/wwdhnctx57b_vhf5wxchq4h00000gn/T/ipykernel_12610/522024696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datensatz_f['weather_condition'] = datensatz_f['weather_condition'].replace({


Vorverarbeitung der Trainingsdaten

In [11]:

datensatz_train, datensatz_test = train_test_split(datensatz_f, test_size=0.1, shuffle=True)

datensatz_mean = datensatz_train.groupby(['hour', 'streetname', 'weekday']).agg({
    'n_pedestrians': 'mean',
    'n_pedestrians_towards': 'mean',
    'n_pedestrians_away': 'mean'
        }).reset_index()

# Index sicherstellen
datensatz_mean = datensatz_mean.reset_index(drop=True)

# Multiindex erstellen
datensatz_mean.set_index(['hour', 'streetname', 'weekday'], inplace=True, drop=False)


Saisonales Vergleichsmodell

In [14]:
from sklearn.metrics import mean_squared_error
from pandas.tseries.offsets import DateOffset

# Tatsächliche Werte und vorhergesagte Werte extrahieren
actual_values = []
predicted_values = []

# Iteriere über jede Zeile des Datensatzes
for index in range(len(datensatz_train)):
    current_data = datensatz_train.iloc[index]

    previous_data = datensatz_mean.loc[(current_data['hour'], current_data['streetname'], current_data['weekday'])]
    
    # Vergleich der Stunden von current_data und previous_data
    assert current_data['hour'] == previous_data['hour'], "Hours do not match between current and previous data"
    
    # Speichern der Vorhersagen
    predicted_values.append([previous_data['n_pedestrians'], previous_data['n_pedestrians_towards'], previous_data['n_pedestrians_away']])
    actual_values.append([current_data['n_pedestrians'], current_data['n_pedestrians_towards'], current_data['n_pedestrians_away']])


# MSE berechnen 
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)

# Ausgabe der Fehlerraten
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")



Mean Squared Error (MSE): 108037.9584269591
Root Mean Squared Error (RMSE): 328.6912813370003
